In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import wandb
import tqdm
import plotly
from plotly.subplots import make_subplots
from scipy.signal import savgol_filter

## Experiment 1

In [ ]:
api = wandb.Api()

# get the runs
runs =  list(api.runs("nauqs/experiment-1")) + list(api.runs("nauqs/experiment-1-b"))

len(runs)

In [ ]:
metrics = ["success_rate", "average_length", "average_return"]

# list to store DataFrames for each run
df_list = []
for run in tqdm.tqdm(runs):
    scan_hist = run.scan_history(keys=["timestep", *metrics], page_size=10000000, min_step=0)
    run_df = pd.DataFrame([val for val in scan_hist])
    run_df["run_id"] = run.id
    df_list.append(run_df)

# concatenate all DataFrames
metrics_df = pd.concat(df_list, ignore_index=True)
config = pd.DataFrame([r.config for r in runs])
config["run_id"] = [r.id for r in runs]
metrics_df = metrics_df.merge(config[["run_id", "final_reward_penalty", "time_cost", "env_id", "gae_lambda", "total_timesteps"]], on="run_id")
metrics_df["agent"] = metrics_df["time_cost"].apply(lambda x: "C" if x > 0 else "A")
metrics_df["agent"] = metrics_df.apply(lambda x: "B" if x["final_reward_penalty"] else x["agent"], axis=1)

In [ ]:
metrics_to_plot = ["success_rate", "average_length", "average_return"]
metrics_legend = {"success_rate": "Success rate", "average_length": "Episode length", "average_return": "Episode return"}
agent_legend = {"A": "A (standard 1/0)", "B": "B (final penalty)", "C": "C (time cost)"}

colors = {"A": "rgba(128, 128, 128, 0.1)", "C": "rgba(221, 167, 0, 0.1)", "B": "rgba(40, 40, 150, 0.1)"}
line_colors = {"A": "rgb(128, 128, 128)", "C": "rgb(221, 167, 0)", "B": "rgb(20, 20, 200)" }

def plot_metrics(env_id, yaxis_max, max_timestep=1000000):

    env_df = metrics_df[metrics_df["env_id"]==env_id]

    # compute median and std for each metric grouped by agent column
    env_df = env_df[["agent", "timestep"] + metrics].groupby(["timestep", "agent"]).agg(["median", "std"]).reset_index()

    # Additional submetrics
    submetrics = ["median", "std"]

    # use plotly to plot the data
    fig = make_subplots(
        rows=1,
        cols=len(metrics_to_plot),
        subplot_titles=[metrics_legend[metric] for metric in metrics_to_plot],
        horizontal_spacing=0.05,
    )

    for i, metric in enumerate(metrics_to_plot):
        for agent in ["A","B", "C"]:
            agent_df = env_df[env_df["agent"] == agent]
            showlegend = True if i == 0 else False 

            # median
            fig.add_trace(
                plotly.graph_objects.Scatter(
                    x=agent_df["timestep"],
                    y=agent_df[metric]["median"],
                    line=dict(color=line_colors[agent]),
                    name=f"{agent_legend[agent]}",
                    mode='lines',
                    line_shape='spline',   
                    showlegend=showlegend  
                ),
                row=1,
                col=i+1,
            )

            # std area
            fig.add_trace(
                plotly.graph_objects.Scatter(
                    x=list(agent_df["timestep"]) + list(agent_df["timestep"])[::-1],
                    y=list(agent_df[metric]["median"]-agent_df[metric]["std"]) + list(agent_df[metric]["median"]+agent_df[metric]["std"])[::-1],
                    fill='toself',
                    fillcolor=colors[agent],
                    line_color='rgba(255,255,255,0)',
                    showlegend=False,
                    mode='lines',
                    line_shape='spline',   
                ),
                row=1,
                col=i+1,
            )
            
        fig.update_yaxes(range=[-0.02*(yaxis_max[metric]), yaxis_max[metric]], row=1, col=i+1)
        fig.update_xaxes(range=[0, max_timestep], row=1, col=i+1)

    fig.update_layout(
        title=env_id,
        xaxis_title="Timestep",
        legend_title="",
        font=dict(size=14),
        width=1200,
        height=350,
        title_x=0.5,
        legend=dict(orientation="h", yanchor="bottom", y=-0.32,xanchor="right",x=0.66), 
        #plot_bgcolor='rgba(255,255,255,1)',        
    )

    fig.show()

    fig.write_image(f"results-{env_id}.png", scale=2)


In [ ]:
def plot_metrics_multiplot(envs, filename="results-exp1-multiplot.png"):

    # use plotly to plot the data
    fig = make_subplots(
        rows=len(envs),
        cols=len(metrics_to_plot),
        subplot_titles=[metrics_legend[metric] for _, _, _ in envs for metric in metrics_to_plot],
        horizontal_spacing=0.05,
        vertical_spacing=0.07,
    )

    for j, (env_id, yaxis_max, max_timestep) in enumerate(envs):
        env_df = metrics_df[metrics_df["env_id"]==env_id]

        # compute median and std for each metric grouped by agent column
        env_df = env_df[["agent", "timestep"] + metrics].groupby(["timestep", "agent"]).agg(["median", "std"]).reset_index()

        for i, metric in enumerate(metrics_to_plot):
            for agent in ["A","B", "C"]:
                agent_df = env_df[env_df["agent"] == agent]
                showlegend = True if i == 0 and j == 0 else False 

                # median
                fig.add_trace(
                    plotly.graph_objects.Scatter(
                        x=agent_df["timestep"],
                        y=agent_df[metric]["median"],
                        line=dict(color=line_colors[agent]),
                        name=f"{agent_legend[agent]}",
                        mode='lines',
                        line_shape='spline',   
                        showlegend=showlegend 
                    ),
                    row=j+1,
                    col=i+1,
                )

                # std area
                fig.add_trace(
                    plotly.graph_objects.Scatter(
                        x=list(agent_df["timestep"]) + list(agent_df["timestep"])[::-1],
                        y=list(agent_df[metric]["median"]-agent_df[metric]["std"]) + list(agent_df[metric]["median"]+agent_df[metric]["std"])[::-1],
                        fill='toself',
                        fillcolor=colors[agent],
                        line_color='rgba(255,255,255,0)',
                        showlegend=False,
                        mode='lines',
                        line_shape='spline',   
                    ),
                    row=j+1,
                    col=i+1,
                )
                
            fig.update_yaxes(range=[-0.02*(yaxis_max[metric]), yaxis_max[metric]], row=j+1, col=i+1)
            fig.update_xaxes(range=[0, max_timestep], row=j+1, col=i+1, title="Timestep" if j == len(envs) - 1 else None)
            

    # Add vertical titles
    annotations = []
    for j, (env_id, _, _) in enumerate(envs):
        annotations.append(dict(xref='paper', yref='paper', x=-0.05, y=(1.1*(len(envs)-j-0.5)/len(envs))-0.05,
                                xanchor='center', yanchor='middle',
                                text=env_id.split("MiniGrid-")[1], showarrow=False,
                                textangle=-90, font=dict(size=18)))
        
    # Add metric subtitles to the first row
    for i, metric in enumerate(metrics_to_plot):
        annotations.append(dict(xref='paper', yref='paper', x=(i+0.5)/len(metrics_to_plot), y=1.01,
                                xanchor='center', yanchor='bottom',
                                text=metrics_legend[metric], showarrow=False,
                                font=dict(size=18)))


    fig.update_layout(
        title="",
        legend_title="",
        font=dict(size=14),
        width=1200,
        height=250*len(envs),
        title_x=0.5,
        legend=dict(orientation="h", yanchor="bottom", y=-0.12,xanchor="right",x=0.70),
        annotations=annotations,
        #plot_bgcolor='rgba(255,255,255,1)',        
    )

    fig.update_layout(margin=dict(l=70, r=0, b=0, t=50))

    fig.show()

    fig.write_image(filename, scale=2)


In [ ]:
envs = [("MiniGrid-Empty-16x16-v0", {"average_length": 500, "average_return": 1.02, "success_rate": 1.02}, 500000),
        ("MiniGrid-SimpleCrossingS9N2-v0", {"average_length": 200, "average_return": 1.02, "success_rate": 1.02}, 1500000),
        ("MiniGrid-FourRooms-v0", {"average_length": 1024, "average_return": 1.02, "success_rate": 1.02}, 5000000)]

plot_metrics_multiplot(envs)


In [ ]:
metrics_df.env_id.unique()

In [ ]:
envs = [("MiniGrid-Unlock-v0", {"average_length": 300, "average_return": 1.02, "success_rate": 1.02}, 800000),
        ("MiniGrid-DoorKey-8x8-v0", {"average_length": 650, "average_return": 1.02, "success_rate": 1.02}, 1500000),
        ("MiniGrid-KeyCorridorS3R2-v0", {"average_length": 300, "average_return": 1.02, "success_rate": 1.02}, 3000000)]

plot_metrics_multiplot(envs, "results-exp1b-multiplot.png")


In [ ]:
runs_mc =  list(api.runs("nauqs/experiment-1-mc"))
len(runs_mc)

In [ ]:
metrics = ["success_rate", "average_length", "average_return"]

df_list = []
for run in tqdm.tqdm(runs_mc):
    scan_hist = run.scan_history(keys=["timestep", *metrics], page_size=10000000, min_step=0)
    run_df = pd.DataFrame([val for val in scan_hist])
    run_df["run_id"] = run.id
    df_list.append(run_df)

mc_df = pd.concat(df_list, ignore_index=True)
config = pd.DataFrame([r.config for r in runs_mc])
config["run_id"] = [r.id for r in runs_mc]
mc_df = mc_df.merge(config[["run_id", "final_reward_penalty", "time_cost", "env_id", "gae_lambda", "seed", "total_timesteps"]], on="run_id")
mc_df["agent"] = mc_df["time_cost"].apply(lambda x: "C" if x > 0 else "A")
mc_df["agent"] = mc_df.apply(lambda x: "B" if x["final_reward_penalty"] else x["agent"], axis=1)

In [404]:
def lambda_multiplot(env, metric, lambda_list, filename="results-lambda-multiplot.png"):
    # use plotly to plot the data
    fig = make_subplots(
        rows=1,
        cols=len(lambda_list),
        subplot_titles=[f"λ={lmbda}" for lmbda in lambda_list],
        horizontal_spacing=0.05,
        vertical_spacing=0.07,
    )

    env_id, max_timesteps, yaxis_max = env
    env_df = mc_df[(mc_df["env_id"] == env_id)&(mc_df["seed"]<=20)]

    for i, lmbda in enumerate(lambda_list):
        lambda_df = env_df[env_df["gae_lambda"] == lmbda]

        # compute median and std for the metric grouped by agent column
        lambda_df = lambda_df[["agent", "timestep", metric]].groupby(["timestep", "agent"]).agg(["median", "std"]).reset_index()

        for agent in ["A", "B", "C"]:
            agent_df = lambda_df[lambda_df["agent"] == agent]
            showlegend = True if i == 0 else False

            # median
            fig.add_trace(
                plotly.graph_objects.Scatter(
                    x=agent_df["timestep"],
                    y=agent_df[metric]["median"],
                    line=dict(color=line_colors[agent]),
                    name=f"{agent_legend[agent]}",
                    mode='lines',
                    line_shape='spline',
                    showlegend=showlegend
                ),
                row=1,
                col=i+1,
            )

            #std
            fig.add_trace(
                plotly.graph_objects.Scatter(
                    x=list(agent_df["timestep"]) + list(agent_df["timestep"])[::-1],
                    y=list(agent_df[metric]["median"]-agent_df[metric]["std"]) + list(agent_df[metric]["median"]+agent_df[metric]["std"])[::-1],
                    fill='toself',
                    fillcolor=colors[agent],
                    line_color='rgba(255,255,255,0)',
                    showlegend=False,
                    mode='lines',
                    line_shape='spline',   
                ),
                row=1,
                col=i+1,
            )

        fig.update_yaxes(range=[-0.02 * (yaxis_max), yaxis_max], row=1, col=i+1)
        fig.update_xaxes(range=[0, max_timesteps[lmbda]], row=1, col=i+1, title="Timestep")

    fig.update_layout(
        title="Episode length (number of timesteps)",
        legend_title="",
        font=dict(size=14),
        width=1200,
        height=350,
        title_x=0.5,
        legend=dict(orientation="h", yanchor="bottom", y=-0.5, xanchor="right", x=0.70),
        # plot_bgcolor='rgba(255,255,255,1)',
        margin=dict(l=20, r=20, b=0, t=80)
    )

    fig.show()

    fig.write_image(filename, scale=2)


In [405]:
lambda_multiplot(env=("MiniGrid-Empty-16x16-v0", 
                        {0.5: 1000000, 0.8: 1000000, 0.9: 500000, 0.95: 500000, 1.0: 500000}, 
                        1050),
                metric="average_length",
                lambda_list=[1.0, 0.95, 0.9, 0.8, 0.5])